In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import sys
sys.path.append('..')

In [3]:
from scraper import Scraper

In [4]:
with open("hypos.txt") as f:
    hypos = [line.strip() for line in f]

with open("refs.txt") as f:
    refs = [line.strip() for line in f]

In [5]:
refs

['https://blog.topcv.vn/co-nen-hoc-nganh-dien-tu-vien-thong/',
 'https://baoantelecom.com/vi/tin/dien-tu-vien-thong',
 'https://hust.edu.vn/vi/dao-tao/ht-dien-tu-thong-minh-internet-van-vat/elitech-dien-tu-vien-thong-364255.html',
 'https://huongnghiep.hocmai.vn/chuyen-nganh/ky-thuat-dien-tu-vien-thong/dai-hoc-bach-khoa-ha-noi/']

In [6]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"

In [7]:
refs = Scraper(refs, user_agent).run()

In [8]:
refs

[{'url': 'https://blog.topcv.vn/co-nen-hoc-nganh-dien-tu-vien-thong/',
  'raw_content': 'Có nên học ngành kỹ thuật điện tử viễn thông hay không?\n16 nghìn người/năm là nhu cầu tuyển dụng lao động làm việc ngành kỹ thuật điện tử viễn thông trong giai đoạn từ 2020 – 2025. Điều này chứng minh, nhân lực ngành này đang tuyển dụng nhiều và có xu hướng tăng qua các năm. Vậy ngành kỹ thuật điện tử viễn thông là gì? Học ngành này ở đâu? Thi những môn nào? Cơ hội việc làm và mức thu nhập ra sao sau khi ra trường? Tất cả sẽ được TopCV cung cấp chi tiết trong bài viết sau đây để bạn cùng tìm hiểu, từ đó dễ dàng đưa ra sự lựa chọn.\nNội dung bài viết\nNgành kỹ thuật điện tử viễn thông là gì?\nNgành kỹ thuật điện tử viễn thông là ngành học đào tạo nguồn nhân lực có chuyên môn và kỹ năng về việc thiết kế, phát triển và vận hành các hệ thống và công nghệ liên quan đến viễn thông và truyền thông điện tử.\nĐây là ngành tập trung vào các công nghệ và ứng dụng sử dụng trong việc truyền tải, xử lý và quản 

In [9]:
len(refs)

4

In [10]:
refs = [item['raw_content'] for item in refs]

In [11]:
refs

['Có nên học ngành kỹ thuật điện tử viễn thông hay không?\n16 nghìn người/năm là nhu cầu tuyển dụng lao động làm việc ngành kỹ thuật điện tử viễn thông trong giai đoạn từ 2020 – 2025. Điều này chứng minh, nhân lực ngành này đang tuyển dụng nhiều và có xu hướng tăng qua các năm. Vậy ngành kỹ thuật điện tử viễn thông là gì? Học ngành này ở đâu? Thi những môn nào? Cơ hội việc làm và mức thu nhập ra sao sau khi ra trường? Tất cả sẽ được TopCV cung cấp chi tiết trong bài viết sau đây để bạn cùng tìm hiểu, từ đó dễ dàng đưa ra sự lựa chọn.\nNội dung bài viết\nNgành kỹ thuật điện tử viễn thông là gì?\nNgành kỹ thuật điện tử viễn thông là ngành học đào tạo nguồn nhân lực có chuyên môn và kỹ năng về việc thiết kế, phát triển và vận hành các hệ thống và công nghệ liên quan đến viễn thông và truyền thông điện tử.\nĐây là ngành tập trung vào các công nghệ và ứng dụng sử dụng trong việc truyền tải, xử lý và quản lý thông tin qua các mạng điện tử và không dây. Trong đó có kể đến các thiết bị truyền 

In [12]:
from unidecode import unidecode
import re

def normalize_text(text):
    """
    Normalize Vietnamese text by converting it to its ASCII representation and removing accents.
    
    Args:
    text (str): Input Vietnamese text to be normalized.
    
    Returns:
    str: Normalized text with accents removed.
    """
    normalized_text = unidecode(text)
    normalized_text = re.sub(r'\s+', ' ', normalized_text).strip()
    return normalized_text


In [13]:
hypos = [' '.join(normalize_text(hypo) for hypo in hypos)]

In [14]:
hypos

['# Co Hoi Viec Lam Cho Sinh Vien Nganh Dien Tu - Vien Thong Cua Dai Hoc Bach Khoa Ha Noi Sau Khi Tot Nghiep  ## Tong Quan  Trong boi canh cuoc Cach mang cong nghiep lan thu 4 dang dien ra manh me, nganh Dien tu - Vien thong dong vai tro trong yeu trong su phat trien cua xa hoi hien dai. Dai hoc Bach khoa Ha Noi (HUST), mot trong nhung truong dai hoc hang dau Viet Nam, da va dang dao tao ra nhung ky su Dien tu - Vien thong co chat luong cao, dap ung nhu cau ngay cang tang cua thi truong lao dong trong va ngoai nuoc.  ## Co Hoi Viec Lam Sau Tot Nghiep  ### Thi Truong Lao Dong  Theo thong ke tu Trung tam Du bao nhu cau nhan luc va Thong tin thi truong lao dong TPHCM, giai doan tu 2020 - 2025, nhu cau tuyen dung lao dong lam viec nganh Dien tu vien thong la khoang 16 nghin nguoi/nam, voi xu huong tang qua cac nam sau do (TopCV). Dieu nay cho thay tiem nang viec lam rong lon cho sinh vien sau khi tot nghiep.  ### Vi Tri Viec Lam  Sinh vien tot nghiep nganh Dien tu - Vien thong tu HUST co t

In [15]:
refs = [normalize_text(ref) for ref in refs]

In [16]:
refs

['Co nen hoc nganh ky thuat dien tu vien thong hay khong? 16 nghin nguoi/nam la nhu cau tuyen dung lao dong lam viec nganh ky thuat dien tu vien thong trong giai doan tu 2020 - 2025. Dieu nay chung minh, nhan luc nganh nay dang tuyen dung nhieu va co xu huong tang qua cac nam. Vay nganh ky thuat dien tu vien thong la gi? Hoc nganh nay o dau? Thi nhung mon nao? Co hoi viec lam va muc thu nhap ra sao sau khi ra truong? Tat ca se duoc TopCV cung cap chi tiet trong bai viet sau day de ban cung tim hieu, tu do de dang dua ra su lua chon. Noi dung bai viet Nganh ky thuat dien tu vien thong la gi? Nganh ky thuat dien tu vien thong la nganh hoc dao tao nguon nhan luc co chuyen mon va ky nang ve viec thiet ke, phat trien va van hanh cac he thong va cong nghe lien quan den vien thong va truyen thong dien tu. Day la nganh tap trung vao cac cong nghe va ung dung su dung trong viec truyen tai, xu ly va quan ly thong tin qua cac mang dien tu va khong day. Trong do co ke den cac thiet bi truyen thong

In [17]:
import textwrap

def wraptext(documents, width=120):
    for idx, doc in enumerate(documents, start=1):
        print(f"Document {idx}:")
        wrapped_lines = textwrap.wrap(doc, width=width)
        for line in wrapped_lines:
            print(line)
        print("-" * width)

In [18]:
print(wraptext(refs))

Document 1:
Co nen hoc nganh ky thuat dien tu vien thong hay khong? 16 nghin nguoi/nam la nhu cau tuyen dung lao dong lam viec nganh
ky thuat dien tu vien thong trong giai doan tu 2020 - 2025. Dieu nay chung minh, nhan luc nganh nay dang tuyen dung
nhieu va co xu huong tang qua cac nam. Vay nganh ky thuat dien tu vien thong la gi? Hoc nganh nay o dau? Thi nhung mon
nao? Co hoi viec lam va muc thu nhap ra sao sau khi ra truong? Tat ca se duoc TopCV cung cap chi tiet trong bai viet sau
day de ban cung tim hieu, tu do de dang dua ra su lua chon. Noi dung bai viet Nganh ky thuat dien tu vien thong la gi?
Nganh ky thuat dien tu vien thong la nganh hoc dao tao nguon nhan luc co chuyen mon va ky nang ve viec thiet ke, phat
trien va van hanh cac he thong va cong nghe lien quan den vien thong va truyen thong dien tu. Day la nganh tap trung vao
cac cong nghe va ung dung su dung trong viec truyen tai, xu ly va quan ly thong tin qua cac mang dien tu va khong day.
Trong do co ke den cac thiet bi tr

In [19]:
len(hypos), len(refs)

(1, 4)

### Using BERTScore for Evaluation

1. BERTScore is used to measure textual similarity between candidate texts and reference texts. It considers not only exact word matches but also the overall meaning, fluency, and order of the ouput.

2. BERTScore: Precision, Recall, F1
    * Precision measures how well the candidate texts avoid introducing irrelevant content.
    * Recall measures how well the candidate texts avoid omitting relevant content.
    * F1 = 2 x (P x R)/(P + R) 

In [20]:
from bert_score import score

/media/sanslab/Data/stevehoang/rag_research/vigpt-researcher/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Hide the loading messages
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [22]:
def bert_score(hypos, refs, lang="vi"):
    bert_scores = []
    for ref in refs:
        scores = score(hypos, [ref], lang=lang, verbose=False)
        bert_scores.append(scores)
    return bert_scores

In [23]:
bert_scores = bert_score(hypos, refs)

In [24]:
import torch

precision = torch.mean(torch.stack([bert_score[0] for bert_score in bert_scores]))
recall = torch.mean(torch.stack([bert_score[1] for bert_score in bert_scores]))
f1 = torch.mean(torch.stack([bert_score[2] for bert_score in bert_scores]))

In [25]:
print("BERT score:")
print(f"P: {precision.item():.2f}, R: {recall.item():.2f}, F1: {f1.item():.2f}")

BERT score:
P: 0.74, R: 0.77, F1: 0.75


### Using ROUGE score for Evaluation

In [26]:
from rouge_score import rouge_scorer

def rouge_score(hypos, refs):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = [scorer.score(ref, hypo) for ref, hypo in zip(refs, hypos)]
    return scores

rouge = rouge_score(hypos, refs)

In [27]:
for i, score in enumerate(rouge):
    print("ROUGE scores:")
    print(f"ROUGE-1: P = {score['rouge1'].precision:.2f}, R = {score['rouge1'].recall:.2f}, F1 = {score['rouge1'].fmeasure:.2f}")
    print(f"ROUGE-2: P = {score['rouge2'].precision:.2f}, R = {score['rouge2'].recall:.2f}, F1 = {score['rouge2'].fmeasure:.2f}")

ROUGE scores:
ROUGE-1: P = 0.89, R = 0.17, F1 = 0.29
ROUGE-2: P = 0.51, R = 0.10, F1 = 0.16


### RESULTS

| METRIC   | Precision | Recall | F1 |
|---------|----------|-------|-------|
| BERTScore| | | |
| ROUGE-1| | | |
| ROUGE-2| | | |

* BERTScore Average: Precision = ?, Recall = ?, F1 = ?
* ROUGE-1 Average: Precision = ?, Recall = ?, F1 = ?
* ROUGE-2 Average: Precision = ?, Recall = ?, F1 = ?

### Total Time Executions

|Time (s) | T1 | T2 |
|----|----|----|
|    | 100.123 | 123.123 |

### Review
1. Summarization is a complex task, even advanced models can struggle to accurately capture all the important information from a source text.

2. BERTScore or ROUGE that these metrics focus on things like including keywords from the source text, so that do not perfectly reflect human judgement of a good summary.

3. The response of chatbot from many different sources.


### HUMAN EVALUATION IS THE BEST WAY TO EVALUATE THE QUALITY OF A SUMMARY